#### Dependents


In [ ]:
!pip install ../input/mmdetection/addict-2.4.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/yapf-0.31.0-py2.py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/terminaltables-3.1.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/einops-0.4.1-py3-none-any.whl > /dev/null
!pip install ../input/mmsegmentation/mmcv-full/mmcv_full-1.5.3-cp37-cp37m-linux_x86_64.whl > /dev/null
!pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/mmcls-0.23.1-py2.py3-none-any.whl > /dev/null

In [ ]:
!cp -r ../input/mmsegm/mmsegmentation-master /kaggle/working/ && cd /kaggle/working/mmsegmentation-master && pip install -e . && cd ..  > /dev/null

#### Import librarys

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from glob import glob
from tqdm.notebook import tqdm
import sys
import gc
sys.path.append('./mmsegmentation-master')
from mmseg.apis import init_segmentor, inference_segmentor
from mmcv.utils import config

#### Load models

In [ ]:

configs = [
    '../input/lastmulti/256mmseg.py',    
    '../input/lastmulti/384mmseg.py',
    '../input/lastmulti/512mmseg.py',
]

ckpts = [
     '../input/lastmulti/256_epoch_140.pth',    
     '../input/lastmulti/384_epoch_20.pth',    
     '../input/lastmulti/512_epoch_90.pth',   
]

DATA = '../input/hubmap-organ-segmentation/test_images/'
df_sample = pd.read_csv('../input/hubmap-organ-segmentation/sample_submission.csv').set_index('id')


models = []
for idx,(cfg, ckpt) in enumerate(zip(configs, ckpts)):
    cfg = config.Config.fromfile(cfg)
    model = init_segmentor(cfg, ckpt, device='cuda:0')
    models.append(model)

#### mask -> rle

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

#### My inference

In [ ]:
df_sample = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')

In [ ]:
from collections import Counter

names,preds = [],[]
imgs,pd_mks = [],[]
debug = len(df_sample)<2

for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):

    idx = str(row['id'])
    img  = cv2.imread(os.path.join(DATA,str(idx)+'.tiff'))
        

    pred = []

    for model in models :
        predict = inference_segmentor(model, img)[0]
        predict_f = predict.flatten()
        cnt = Counter(predict_f)
        if cnt.most_common(1)[0][0] != 0 :
            max_num = cnt.most_common(1)[0][0]
            predict = (predict == max_num).astype(np.uint8)
            pred.append(predict)

        else :
            if len(cnt.most_common()) > 1 :
                max_num = cnt.most_common(2)[1][0]
                predict = (predict == max_num).astype(np.uint8)
                pred.append(predict)
            else : pred.append(predict)

        pred = (sum(pred) / len(pred))
        pred = (pred>0.25).astype(np.uint8)
        # pred = np.ceil((sum(pred) / len(pred))).astype(np.uint8)

        rle = rle_encode_less_memory(pred)
        names.append(str(idx))
        preds.append(rle)
        
        if debug:
            imgs.append(img)
            pd_mks.append(pred)
        del img, pred, rle, idx, row

In [ ]:
if debug:
    import matplotlib.pyplot as plt
    for img, mask in zip(imgs, pd_mks):
        plt.figure(figsize=(12, 7))
        plt.subplot(1, 3, 1); plt.imshow(img); plt.axis('OFF'); plt.title('image')
        plt.subplot(1, 3, 2); plt.imshow(mask*255); plt.axis('OFF'); plt.title('mask')
        plt.subplot(1, 3, 3); plt.imshow(img); plt.imshow(mask*255, alpha=0.4); plt.axis('OFF'); plt.title('overlay')
        plt.tight_layout()
        plt.show()

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
df = pd.DataFrame({'id':names,'rle':preds})
df.to_csv('submission.csv',index=False)

In [ ]:
df.head()